In [2]:
#import keras
#from keras.application import vgg16
import mysql.connector
from mysql.connector import errorcode

config = {
    'user': 'root',
    'password': 'GETFILE',
    'host': '35.193.20.163',
    'database': 'usr'
}


from PIL import Image, ImageDraw, ImageFont
from matplotlib.pyplot import imshow
import pytesseract, cv2
import numpy as np
import pytesseract
import time
import pyodbc
%matplotlib inline

In [3]:
def test(s):
    img = Image.open(s)
    return pytesseract.image_to_string(s)

def testrun():
    return test('license.jpg')

In [4]:
def processImage(img, exp = 128):
    w, h = img.size
    #x, y, wcrop, hcrop = 120, 78, 50, 50
    cropped_image = img#.crop((x, y, w - wcrop, h - hcrop))
    gray = cropped_image.convert('L')
    bw = gray.point(lambda x: 0 if x<exp else 255, '1')
    cropped_image = bw.convert('RGBA')
    return cropped_image
    
def show(i):
    imshow(np.asarray(i))

In [ ]:
img = Image.open('license.jpg').convert('RGBA')
w, h = img.size

In [ ]:
x, y, wcrop, hcrop = 120, 78, 50, 50
cropped_image = img.crop((x, y, w - wcrop, h - hcrop))
#show(cropped_image)
gray = cropped_image.convert('L')
bw = gray.point(lambda x: 0 if x<38 else 255, '1')
show(bw.convert('RGBA'))
cropped_image = bw.convert('RGBA')

In [ ]:
boxes = pytesseract.image_to_boxes(cropped_image)
pytesseract.image_to_string(cropped_image)

In [ ]:
bout = Image.new('RGBA', cropped_image.size, (255,255,255,0))
d = ImageDraw.Draw(bout)
#[[_[0]] + [int(__) for __ in _.split()[1:]] for _ in boxes.split('\n')[1:]]
[d.rectangle([int(__) for __ in _.split()[1:-1]]) for _ in boxes.split('\n')[:]]
d.text((100, 10), "Hello", fill=(255, 0, 0, 255))

In [ ]:
out = Image.alpha_composite(cropped_image, bout)
show(out)

In [5]:
def get_plate():
    refresh_rate = 750
    guesses, best_guess = {}, ""
    gcount, gmax = 0, 25
    fnt = ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 40)
    x, t = 0, time.clock()
    cap = cv2.VideoCapture (0)
    while (True):
        ret, frame = cap.read()
        if not ret:
            continue
        gray = cv2.cvtColor (frame, cv2.COLOR_BGR2GRAY)
        gray = processImage(Image.fromarray(gray))

        if x%refresh_rate:
            txt = pytesseract.image_to_string(gray)
            guesses.update({txt: guesses[txt] + 1 if txt in guesses else 1})
            if txt is not '':
                gcount += 1

            bout = Image.new('RGBA', gray.size, (255,255,255,0))
            bw, bh = bout.size
            d = ImageDraw.Draw(bout)
            d.text((10,10), ascii(txt), font=fnt, fill=(255,0,0,255))

            best_guess = make_guess(guesses)
            if valid_guess(best_guess):
                d.text((10, bh - 50), best_guess, font=fnt, fill=(0,255,0,255))
            else:
                d.text((10, bh - 50), best_guess, font=fnt, fill=(255,0,0,255))
            gray = Image.alpha_composite(gray, bout)

            #print(1/(time.clock() - t))
            t = time.clock()
            if gcount > gmax:
                gcount = 0
                guesses = {}

        cv2.imshow('frame', np.array(gray.convert('RGB'))[:, :, ::-1].copy())
        if cv2.waitKey (1) & 0xFF == ord('q'):
            return best_guess
        x += 1
        if len(best_guess) > 1 and valid_guess(best_guess):
            return best_guess

In [6]:
def make_guess(g):
    km, vm = '', 0
    for k, v in g.items():
        if k is not '' and v > 1:
            if v > vm:
                vm = v
                km = k
    return km

def valid_guess(s):
    domain = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-'
    for _ in s:
        if _ not in domain:
            return False
    return True

In [13]:
guess = get_plate()
cnx = cur = None
try:
    cnx = mysql.connector.connect(**config)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print('Something is wrong with your user name or password')
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cur = cnx.cursor()
    #cur.execute('show databases;')
    #for row in cur.fetchall():
    #    print(row)
    
    cur.execute('USE usr;')
    cur.execute('SELECT * FROM user WHERE plate_num = "{}";'.format(guess))
    for row in cur.fetchall():
        print(row)
finally:
    if cur:
        cur.close()
    if cnx:
        cnx.close()

ModuleNotFoundError: No module named 'date'